# Tutorial

I utilize a package called fast-arrow, which was developed by Weston Platter. It's a Robinhood API wrapper. It's great. Unfortunately, it can be a little finicky. If you have any problems, just message me and I will help figure it out. But basically, all we need acess to is options chain data. In the tutorial below, I will go through (in more depth) exactly what information we have, how it is structured. 
### Import Packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import getpass
import os
cdir =os.getcwd()
os.chdir('Fat_Tailed_Option_Model')
try:
    from Robinhood_Data_Datafetcher import Robinhood_Data
    from Fat_Tailed_Model import Fat_Tailed_Option_Model
    print('Packages Imported')
    os.chdir(cdir)
except:
    os.chdir(cdir)

from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 un
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

Packages Imported


In [2]:
print('RobinHood Username:')
username = input()
print('RobinHood PW:')
password = getpass.getpass()
data = Robinhood_Data(username,password) 

Username:
thaversang@gmail.com
········
Trys: 1.


### The Robinhood_Data Object
This object `data` allows us to fetch data for a given symbol. We have two methods available. One can fetch the data for a specific symbol on a specific expiration date. The method is called `data.get_options_robinhood`. The other, `data.get_all_options_robinhood` fetches *all* of the expirations. 

A quick note about a small quirk. The method `data.get_options_robinhood` returns just one pandas dataframe, containing the calls and puts together. This can be easily split into calls and puts by using a filter on the `type` column. The method `data.get_all_options_robinhood` returns three objects, which are calls, puts and the spot price. 

Further, the method `data.get_all_options_robinhood` is built on top of DASK, to hasten the datafetching process. Rather than fetch the data one after another in a loop, the data is fetched in parallel. 

In [3]:
option_df = data.get_options_robinhood('SPY',exp = '2019-09-20')
call_df,put_df = option_df.loc[option_df.type =='call'],option_df.loc[option_df.type =='put']
spot_ = call_df.spot_price.iat[0]

In [ ]:
plt.plot(put_df.strike_price,put_df.adjusted_mark_price,'.')
plt.plot(call_df.strike_price,call_df.adjusted_mark_price,'.')
plt.legend(['Put','Call'])

# Examining The Surfaces

In [ ]:
f = puts.loc[(puts.bid_size>0) & (puts.ask_size >0)]
f.expiration_date = f.expiration_date.astype(np.int)
X,Y,Z = get_surface(f.copy(),'strike_price','expiration_date','delta')

In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')
surf = ax.plot_surface(X, Y, Z, cmap='bone_r',linewidth=0.5, antialiased=False,edgewidth = 2)
fig.colorbar(surf, shrink=0.5, aspect=5)
plt.show()


### Split the Data Using .Groupby Method
Having all the calls in one frame can be a little overwhelming. We can choose to group based on the expiration date or on the strike, and make meaningful comparisions if we so choose. 

In [ ]:
calls.index = calls.chain_id
puts.index = puts.chain_id
call_strike_group = calls.groupby('strike_price')
call_exp_group = calls.groupby('expiration_date')
puts_strike_group = puts.groupby('strike_price')
puts_exp_group = puts.groupby('expiration_date')

In [ ]:
plt.plot(pd.to_datetime(puts_strike_group.get_group(280).expiration_date),
         puts_strike_group.get_group(280).gamma)




In [ ]:
puts.expiration_date

In [ ]:
call,put, spot = data.get_all_options_robinhood('AXP')

In [ ]:
model = Fat_Tailed_Option_Model(call_option_dataframe=call,put_option_dataframe=put,
                                cutoff_thresh=0.25,
                                evaluation_date='2019-08-17',symbol = 'AFA')



In [ ]:
model.fit_puts(weights='open_interest')
model.fit_calls(weights='open_interest')

In [ ]:
g = model.deep_otm_calls.groupby('expiration_date')
for i in g.groups.keys():
    print(i,len(g.get_group(i)))

In [ ]:
df = g.get_group('2019-08-30')

In [ ]:
plt.figure(figsize =(6,4),dpi = 200)
plt.plot(df.high_fill_rate_sell_price,'-')
plt.plot(df.high_fill_rate_buy_price,'.')
plt.plot(df.model_price,'-o')
print(model.put_alpha)

In [ ]:
model.put_alpha

In [ ]:
#historical returns
spy = pd.read_csv('SPY.csv',index_col = 0)
returns = spy['Adj Close'].pct_change().dropna().sort_values()


In [ ]:
#normal variables, with same mean and standard deviation as historical returns
standard_normal = np.random.normal(size = 1000000)
standard_normal = standard_normal - standard_normal.mean()
standard_normal = standard_normal/standard_normal.std()
normal_vars = pd.Series((standard_normal)*returns.std() + returns.mean())

In [ ]:
#let's simulate_prices...
rvs = np.random.uniform(size = 10**5)
normal_prices = (1+normal_vars.quantile(rvs))*100
normal_prices.index = np.arange(0,len(normal_prices))
historical_prices = (1+returns.quantile(rvs))*100
historical_prices.index = np.arange(0,len(historical_prices))

In [ ]:
# let's sell some OTM puts
def get_profit(strike,spot, premium):
    if strike>spot:
        return (strike - spot)#loss
    else:
        return premium # gain premium

In [ ]:
#get the fair price
strike = 120
historical_fair_price = np.maximum(strike-historical_prices,0).mean()
normal_fair_price = np.maximum(strike-normal_prices,0).mean()

In [ ]:
print('Fair Price from Historical Data ',historical_fair_price)
print('Fair Price from Normal Data ',normal_fair_price)

In [ ]:
normal_pnl = normal_prices.apply(lambda x: get_profit(strike, spot = x,premium = normal_fair_price))
hist_pnl = historical_prices.apply(lambda x: get_profit(strike, spot = x,premium = historical_fair_price))


In [ ]:
normal_pnl.cumsum().plot()
normal_pnl.cumsum().plot()

In [ ]:
normal_prices.plot()